In [1]:
%run utils.ipynb

In [3]:
import pandas as pd
import numpy as np
import os
import sys
# from utils import *
import matplotlib.pyplot as plt
%matplotlib inline
import dill

In [4]:
import sklearn
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, top_k_accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, FunctionTransformer, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from collections import Counter

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation, Input, Embedding, TextVectorization, Reshape, Add, Concatenate, Flatten
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [6]:
TRAIN = 5000
SEG_SIZE = 100
TRAIN_SEG = round(TRAIN / SEG_SIZE)
LABELED_USERS = 10
EMBEDDING_DIM = 128

In [667]:
_, seg_df = load_user_data()
print(seg_df.shape)
y_seg = pd.read_csv('challengeToFill.csv').drop(columns=['Unnamed: 0']).T.reset_index(drop=True).T
y_seg_train = y_seg.iloc[:, :TRAIN_SEG]
y_seg_test = y_seg.iloc[:, TRAIN_SEG:]
train_df = seg_df.iloc[:, :TRAIN_SEG]
test_df = seg_df.iloc[:, TRAIN_SEG:]
submission_df = seg_df.iloc[LABELED_USERS:, TRAIN_SEG:]
# sentences = train_df.to_numpy().flatten().tolist()
# embedding_model = create_embeddings(sentences, vector_size=EMBEDDING_DIM, window=5)
# print(embedding_model.wv)
print(train_df.shape)
print(test_df.shape)

(40, 150)
(40, 50)
(40, 100)


In [668]:
tom = [' '.join(x) for x in train_df.to_numpy().flatten()]

In [669]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,3), min_df=0.01, max_df=0.1, sublinear_tf=True)
mich = pd.DataFrame.sparse.from_spmatrix(tfidf_vec.fit_transform(tom), columns=tfidf_vec.get_feature_names_out())
mich['user'] = pd.Series([idx//TRAIN_SEG for idx in mich.index])
mich

,20 bash,20 bash tcsh,20 csh,20 csh xwsh,20 ls,20 ls mkdir,20 rcp,20 rcp rcp,20 tcsh,20 tcsh tcsh,...,xwsh sh,xwsh sh sh,xwsh xterm,xwsh xterm ksh,yppasswd,yppasswd passwd,yppasswd passwd netscape,yppasswd tbl,yppasswd tbl neqn,user
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39


In [670]:
X_train = mich.sample(frac=1)
y_train_user = X_train['user']
X_train.drop(columns=['user'], inplace=True)

In [671]:
# clf = RandomForestClassifier(random_state=42, max_depth=3, min_samples_leaf=30, min_samples_split=15, max_features='log2')
# clf = MLPClassifier(random_state=42, learning_rate='adaptive', hidden_layer_sizes=(64,))
clf = KNeighborsClassifier(n_neighbors=10, weights='distance')
clf.fit(X_train, y_train_user)

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [672]:
train_preds = clf.predict(X_train)
train_probas = clf.predict_proba(X_train)

In [673]:
top_k_accuracy_score(y_train_user, train_probas, k=1)

1.0

In [674]:
test = [' '.join(x) for x in test_df.to_numpy().flatten()]
X_test = pd.DataFrame.sparse.from_spmatrix(tfidf_vec.transform(test))
y_test_user = pd.Series([idx//(2*TRAIN_SEG) for idx in X_test.index])
X_test

,0,1,2,3,4,5,6,7,8,9,...,2846,2847,2848,2849,2850,2851,2852,2853,2854,2855
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [675]:
test_preds = clf.predict(X_test)
test_probas = clf.predict_proba(X_test)

C:\Users\tom\.conda\envs\dl\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [676]:
top_k_accuracy_score(y_test_user, test_probas, k=1)

0.87325

In [677]:
def get_top_k_probas(probas, k=5):
    return np.partition(probas, -k, axis=1)[:, -k:]

In [678]:
def get_top_k_classes(probas, k=5):
    return np.argpartition(probas, -k, axis=1)[:, -k:]

In [679]:
y_pred_seg = np.array([0 if t in p else 1 for t, p in zip(y_test_user, get_top_k_classes(test_probas, k=3))])
y_pred_seg.shape

(4000,)

In [680]:
y_test = y_seg_test.to_numpy().flatten()

In [681]:
# get_metrics(y_test, y_pred_seg)

In [682]:
clf_name = clf.__class__.__name__
clf_name

'KNeighborsClassifier'

In [683]:
f = FunctionTransformer(lambda y: [' '.join(x) for x in y.to_numpy().flatten()])
pipe = make_pipeline(f, tfidf_vec, clf)
pipe

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function <lambda> at 0x000001BD4F706670>)),
                ('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.1, min_df=0.01, ngram_range=(1, 3),
                                 sublinear_tf=True)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_neighbors=10, weights='distance'))])

In [684]:
pipe.fit(train_df, y_train_user)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function <lambda> at 0x000001BD4F706670>)),
                ('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.1, min_df=0.01, ngram_range=(1, 3),
                                 sublinear_tf=True)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_neighbors=10, weights='distance'))])

In [685]:
y_train_user

969     19
1115    22
1229    24
1960    39
862     17
        ..
347      6
1853    37
831     16
777     15
1517    30
Name: user, Length: 2000, dtype: int64

In [686]:
pipe.predict_proba(test_df)

array([[0.09611604, 0.        , 0.21751165, ..., 0.        , 0.        ,
        0.        ],
       [0.10025313, 0.        , 0.21670999, ..., 0.        , 0.        ,
        0.        ],
       [0.10053304, 0.        , 0.21662379, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.10346519, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.10352867, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.10352867, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [687]:
with open(f'{clf_name}.dill', 'wb') as f:
    dill.dump(pipe, f)

In [688]:
X_train = seg_df.iloc[:, :TRAIN_SEG]
X_train.shape

(40, 50)

In [689]:
func = FunctionTransformer(lambda y: [' '.join(x) for x in y.to_numpy().flatten()])
count_vec = CountVectorizer(lowercase=False)
pipe = make_pipeline(func, count_vec)
tom = pd.DataFrame.sparse.from_spmatrix(pipe.fit_transform(X_train), columns=count_vec.get_feature_names_out()).sparse.to_dense() #.astype(int) #.mode(axis=1)
tom
# tom = tom.astype(np.int32)

,20,4Dwm,9term,FIFO,LOCK,Mail,MediaMai,Sqpe,UNLOCK,Xsession,...,xmodmap,xprop,xrdb,xsessio,xset,xsetroot,xterm,xupdate,xwsh,yppasswd
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,0,0,2,2,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1996,0,0,0,2,2,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1997,0,0,0,2,2,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1998,0,0,0,2,2,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [690]:
from scipy import stats

In [691]:
one_hot_encoder = OneHotEncoder(sparse=False)

In [692]:
unique_transormer = FunctionTransformer(lambda y: y.applymap(lambda x: len(set(x))))

In [693]:
mode_transformer = FunctionTransformer(lambda x: stats.mode(x.to_numpy().flatten().tolist(), axis=1).mode)

In [694]:
mode_count_transformer = FunctionTransformer(lambda x: stats.mode(x.to_numpy().flatten().tolist(), axis=1).count)

In [695]:
X_train = pd.DataFrame(train_df.to_numpy().flatten())
X_train

,0
0,"[cat, nawk, nawk, uname, pwd, echo, echo, ksh,..."
1,"[nawk, nawk, uname, pwd, echo, echo, ksh, unam..."
2,"[nawk, uname, pwd, echo, echo, ksh, uname, stt..."
3,"[uname, pwd, echo, echo, ksh, uname, stty, tse..."
4,"[pwd, echo, echo, ksh, uname, stty, tset, ksh,..."
...,...
1995,"[hostname, id, nawk, getopt, true, true, grep,..."
1996,"[id, nawk, getopt, true, true, grep, date, lp,..."
1997,"[nawk, getopt, true, true, grep, date, lp, fin..."
1998,"[getopt, true, true, grep, date, lp, find, exp..."


In [696]:
longest_word_transformer = FunctionTransformer(lambda df: df.applymap(lambda x: max(x, key=lambda y: len(y))))

In [697]:
len_longest_word_transformer = FunctionTransformer(lambda df: df.applymap(lambda x: len(max(x, key=lambda y: len(y)))))

In [698]:
counter = Counter()
for x in X_train.values.flatten():
    counter.update(x)

In [699]:
counter

Counter({'cat': 9832,
         'nawk': 2883,
         'uname': 2488,
         'pwd': 749,
         'echo': 2240,
         'ksh': 7535,
         'stty': 3294,
         'tset': 1727,
         'movemail': 521,
         'gzip': 21,
         'ls': 10021,
         'sendmail': 7455,
         'grep': 1449,
         'find': 657,
         'mkdir': 675,
         'ln': 1927,
         'xgvis': 384,
         'sh': 19287,
         'Sqpe': 100,
         'whoami': 797,
         'ispell': 33,
         'gs': 412,
         'ghostvie': 307,
         'cpp': 736,
         'xrdb': 828,
         'mkpts': 699,
         'hostname': 5432,
         'env': 724,
         'tcsh': 3545,
         'kill': 246,
         'userenv': 805,
         'wait4wm': 836,
         'xhost': 921,
         'xsetroot': 953,
         'reaper': 1018,
         'mail': 1860,
         'launchef': 7683,
         'netstat': 602,
         'netscape': 9680,
         'rm': 3187,
         'MediaMai': 1868,
         'id': 577,
         'getopt': 44

In [700]:
w2x = {k: idx for idx, k in enumerate(counter.keys())}
x2w = {idx: k for idx, k in enumerate(counter.keys())}
print(w2x)
print(x2w)

{'cat': 0, 'nawk': 1, 'uname': 2, 'pwd': 3, 'echo': 4, 'ksh': 5, 'stty': 6, 'tset': 7, 'movemail': 8, 'gzip': 9, 'ls': 10, 'sendmail': 11, 'grep': 12, 'find': 13, 'mkdir': 14, 'ln': 15, 'xgvis': 16, 'sh': 17, 'Sqpe': 18, 'whoami': 19, 'ispell': 20, 'gs': 21, 'ghostvie': 22, 'cpp': 23, 'xrdb': 24, 'mkpts': 25, 'hostname': 26, 'env': 27, 'tcsh': 28, 'kill': 29, 'userenv': 30, 'wait4wm': 31, 'xhost': 32, 'xsetroot': 33, 'reaper': 34, 'mail': 35, 'launchef': 36, 'netstat': 37, 'netscape': 38, 'rm': 39, 'MediaMai': 40, 'id': 41, 'getopt': 42, 'true': 43, 'date': 44, 'lp': 45, 'expr': 46, 'generic': 47, 'file': 48, 'ppost': 49, 'awk': 50, 'bc': 51, 'postprin': 52, 'hpost': 53, 'getpgrp': 54, 'LOCK': 55, 'sed': 56, 'FIFO': 57, 'download': 58, 'gethost': 59, 'tcpostio': 60, 'csh': 61, 'xterm': 62, 'rlogin': 63, 'tput': 64, 'finger': 65, 'emacs-20': 66, 'xwsh': 67, 'gettxt': 68, 'xconfirm': 69, 'endsessi': 70, 'tellwm': 71, 'xprop': 72, 'xdm': 73, '4Dwm': 74, 'toolches': 75, 'faces': 76, 'xcloc

In [701]:
ct = ColumnTransformer([
    ('mode', mode_transformer, [0]), 
    ('mode_count', mode_count_transformer, [0]), 
    ('unique', unique_transormer, [0]),
    ('longest_word', longest_word_transformer, [0]),
    ('len_longest_word', len_longest_word_transformer, [0])
    ], remainder='passthrough')
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('mode',
                                 FunctionTransformer(func=<function <lambda> at 0x000001BD50116160>),
                                 [0]),
                                ('mode_count',
                                 FunctionTransformer(func=<function <lambda> at 0x000001BD50116430>),
                                 [0]),
                                ('unique',
                                 FunctionTransformer(func=<function <lambda> at 0x000001BD52B924C0>),
                                 [0]),
                                ('longest_word',
                                 FunctionTransformer(func=<function <lambda> at 0x000001BD50143AF0>),
                                 [0]),
                                ('len_longest_word',
                                 FunctionTransformer(func=<function <lambda> at 0x000001BD50143E50>),
                                 [0])])

In [702]:
# X_train.to_numpy().flatten().tolist()

In [703]:
one_hot_encoder = OneHotEncoder(dtype='int')

In [704]:
ordinal_encoder = OrdinalEncoder(dtype='int')

In [705]:
# X_train = pd.DataFrame(X_train.to_numpy().flatten().tolist())
# X_train

In [706]:
ct2 = make_column_transformer((FunctionTransformer(lambda df: [w2x[x[0]] for x in df]), [0, 3]))
ct3 = make_column_transformer((one_hot_encoder, [0, 3]))

In [707]:
pipeline = make_pipeline(ct)
X_train = pd.DataFrame(train_df.to_numpy().flatten())
tom = pd.DataFrame(pipeline.fit_transform(X_train), columns=['mode', 'mode_count', 'num_unique', 'longest_word', 'len_longest_word'])
tom

,mode,mode_count,num_unique,longest_word,len_longest_word
0,sh,15,18,movemail,8
1,sh,15,18,movemail,8
2,sh,15,18,movemail,8
3,ls,15,18,movemail,8
4,sh,16,18,movemail,8
...,...,...,...,...,...
1995,netscape,18,36,hostname,8
1996,netscape,18,35,download,8
1997,netscape,19,34,download,8
1998,netscape,20,33,download,8


In [708]:
tfidf_vec = TfidfVectorizer(ngram_range=(3,3), min_df=0.1, max_df=0.25, sublinear_tf=True, max_features=10)
f = FunctionTransformer(lambda y: [' '.join(x) for x in y.to_numpy().flatten()])
tfidf_pipe = make_pipeline(f, tfidf_vec)
tfidf_df = pd.DataFrame.sparse.from_spmatrix(tfidf_pipe.fit_transform(X_train), columns=tfidf_vec.get_feature_names_out())
tfidf_df

,generic date generic,ls ls ls,ls mc lc,ls sed fifo,sendmail sendmail sendmail,sendmail sh mediamai,sh mediamai sendmail,sh sendmail sendmail,sh sh sh,sh xrdb mkpts
0,0.000000,0.829474,0.0,0.000000,0.0,0.0,0.0,0.0,0.558545,0.0
1,0.000000,0.829474,0.0,0.000000,0.0,0.0,0.0,0.0,0.558545,0.0
2,0.000000,0.829474,0.0,0.000000,0.0,0.0,0.0,0.0,0.558545,0.0
3,0.000000,0.829474,0.0,0.000000,0.0,0.0,0.0,0.0,0.558545,0.0
4,0.000000,0.829474,0.0,0.000000,0.0,0.0,0.0,0.0,0.558545,0.0
...,...,...,...,...,...,...,...,...,...,...
1995,0.713916,0.000000,0.0,0.700231,0.0,0.0,0.0,0.0,0.000000,0.0
1996,0.713916,0.000000,0.0,0.700231,0.0,0.0,0.0,0.0,0.000000,0.0
1997,0.713916,0.000000,0.0,0.700231,0.0,0.0,0.0,0.0,0.000000,0.0
1998,0.713916,0.000000,0.0,0.700231,0.0,0.0,0.0,0.0,0.000000,0.0


In [709]:
tom['mode'] = tom['mode'].apply(lambda x: w2x[x])
tom['longest_word'] = tom['longest_word'].apply(lambda x: w2x[x])
tom

,mode,mode_count,num_unique,longest_word,len_longest_word
0,17,15,18,8,8
1,17,15,18,8,8
2,17,15,18,8,8
3,10,15,18,8,8
4,17,16,18,8,8
...,...,...,...,...,...
1995,38,18,36,26,8
1996,38,18,35,58,8
1997,38,19,34,58,8
1998,38,20,33,58,8


In [710]:
one_hot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
ct3 = make_column_transformer((one_hot_encoder, ['mode', 'longest_word']), (MinMaxScaler(), ['mode_count', 'num_unique', 'len_longest_word']), remainder='passthrough')
tom2 = pd.DataFrame(ct3.fit_transform(tom), columns=ct3.get_feature_names_out())
tom2

,onehotencoder__mode_0,onehotencoder__mode_1,onehotencoder__mode_5,onehotencoder__mode_10,onehotencoder__mode_11,onehotencoder__mode_17,onehotencoder__mode_19,onehotencoder__mode_26,onehotencoder__mode_28,onehotencoder__mode_35,...,onehotencoder__longest_word_190,onehotencoder__longest_word_191,onehotencoder__longest_word_193,onehotencoder__longest_word_204,onehotencoder__longest_word_215,onehotencoder__longest_word_216,onehotencoder__longest_word_217,minmaxscaler__mode_count,minmaxscaler__num_unique,minmaxscaler__len_longest_word
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.126984,0.319149,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.126984,0.319149,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.126984,0.319149,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.126984,0.319149,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.319149,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.174603,0.702128,1.0
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.174603,0.680851,1.0
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.190476,0.659574,1.0
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206349,0.638298,1.0


In [711]:
y_train_user.sort_index()

0        0
1        0
2        0
3        0
4        0
        ..
1995    39
1996    39
1997    39
1998    39
1999    39
Name: user, Length: 2000, dtype: int64

In [712]:
X_test = pd.DataFrame(test_df.to_numpy().flatten())
X_test.shape

(4000, 1)

In [713]:
tom_test = pd.DataFrame(pipeline.transform(X_test), columns=['mode', 'mode_count', 'num_unique', 'longest_word', 'len_longest_word'])
tom_test

,mode,mode_count,num_unique,longest_word,len_longest_word
0,sh,19,16,sendmail,8
1,sh,19,16,sendmail,8
2,sh,19,16,sendmail,8
3,sh,19,16,sendmail,8
4,sh,19,16,sendmail,8
...,...,...,...,...,...
3995,netscape,19,30,MediaMai,8
3996,netscape,20,30,netscape,8
3997,netscape,20,30,netscape,8
3998,netscape,20,30,launchef,8


In [714]:
tom_test['mode'] = tom_test['mode'].apply(lambda x: w2x.get(x, -1))
tom_test['longest_word'] = tom_test['longest_word'].apply(lambda x: w2x.get(x, -1))
# tom_test
tom_test2 = pd.DataFrame(ct3.transform(tom_test), columns=ct3.get_feature_names_out()) #.join(tfidf_df_test)
tom_test2

,onehotencoder__mode_0,onehotencoder__mode_1,onehotencoder__mode_5,onehotencoder__mode_10,onehotencoder__mode_11,onehotencoder__mode_17,onehotencoder__mode_19,onehotencoder__mode_26,onehotencoder__mode_28,onehotencoder__mode_35,...,onehotencoder__longest_word_190,onehotencoder__longest_word_191,onehotencoder__longest_word_193,onehotencoder__longest_word_204,onehotencoder__longest_word_215,onehotencoder__longest_word_216,onehotencoder__longest_word_217,minmaxscaler__mode_count,minmaxscaler__num_unique,minmaxscaler__len_longest_word
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.190476,0.276596,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.190476,0.276596,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.190476,0.276596,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.190476,0.276596,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.190476,0.276596,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.190476,0.574468,1.0
3996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206349,0.574468,1.0
3997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206349,0.574468,1.0
3998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206349,0.574468,1.0


In [715]:
tom_train = tom2.join(tfidf_df)
tom_train

,onehotencoder__mode_0,onehotencoder__mode_1,onehotencoder__mode_5,onehotencoder__mode_10,onehotencoder__mode_11,onehotencoder__mode_17,onehotencoder__mode_19,onehotencoder__mode_26,onehotencoder__mode_28,onehotencoder__mode_35,...,generic date generic,ls ls ls,ls mc lc,ls sed fifo,sendmail sendmail sendmail,sendmail sh mediamai,sh mediamai sendmail,sh sendmail sendmail,sh sh sh,sh xrdb mkpts
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.829474,0.0,0.000000,0.0,0.0,0.0,0.0,0.558545,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.829474,0.0,0.000000,0.0,0.0,0.0,0.0,0.558545,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.829474,0.0,0.000000,0.0,0.0,0.0,0.0,0.558545,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.829474,0.0,0.000000,0.0,0.0,0.0,0.0,0.558545,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.829474,0.0,0.000000,0.0,0.0,0.0,0.0,0.558545,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.713916,0.000000,0.0,0.700231,0.0,0.0,0.0,0.0,0.000000,0.0
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.713916,0.000000,0.0,0.700231,0.0,0.0,0.0,0.0,0.000000,0.0
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.713916,0.000000,0.0,0.700231,0.0,0.0,0.0,0.0,0.000000,0.0
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.713916,0.000000,0.0,0.700231,0.0,0.0,0.0,0.0,0.000000,0.0


In [716]:
mlp = MLPClassifier(learning_rate='adaptive', max_iter=2000, learning_rate_init=0.002, hidden_layer_sizes=(75,), random_state=42)
# mlp = KNeighborsClassifier(n_neighbors=3, weights='distance')
# mlp = RandomForestClassifier(random_state=42, n_estimators=50, min_samples_leaf=2, min_samples_split=5, max_depth=3)
mlp.fit(tom_train, y_train_user)

C:\Users\tom\.conda\envs\dl\lib\site-packages\sklearn\utils\validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(75,), learning_rate='adaptive',
              learning_rate_init=0.002, max_iter=2000, random_state=42)

In [717]:
preds = mlp.predict(tom_train)
probas = mlp.predict_proba(tom_train)

C:\Users\tom\.conda\envs\dl\lib\site-packages\sklearn\utils\validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\tom\.conda\envs\dl\lib\site-packages\sklearn\utils\validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [718]:
tfidf_df_test = pd.DataFrame.sparse.from_spmatrix(tfidf_pipe.transform(X_test), columns=tfidf_vec.get_feature_names_out())
tom_test3 = tom_test2.join(tfidf_df_test)

In [719]:
tom_test3

,onehotencoder__mode_0,onehotencoder__mode_1,onehotencoder__mode_5,onehotencoder__mode_10,onehotencoder__mode_11,onehotencoder__mode_17,onehotencoder__mode_19,onehotencoder__mode_26,onehotencoder__mode_28,onehotencoder__mode_35,...,generic date generic,ls ls ls,ls mc lc,ls sed fifo,sendmail sendmail sendmail,sendmail sh mediamai,sh mediamai sendmail,sh sendmail sendmail,sh sh sh,sh xrdb mkpts
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.348361,0.518081,0.519393,0.344821,0.345873,0.319277
3996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.348361,0.518081,0.519393,0.344821,0.345873,0.319277
3997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.348361,0.518081,0.519393,0.344821,0.345873,0.319277
3998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.348361,0.518081,0.519393,0.344821,0.345873,0.319277


In [720]:
test_preds = mlp.predict(tom_test3)
test_probas = mlp.predict_proba(tom_test3)

C:\Users\tom\.conda\envs\dl\lib\site-packages\sklearn\utils\validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\tom\.conda\envs\dl\lib\site-packages\sklearn\utils\validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0      0
1      0
2      0
3      0
4      0
      ..
495    4
496    4
497    4
498    4
499    4
Length: 500, dtype: int64

In [740]:
y_test_labeld_user = y_test_user[:LABELED_USERS*2*TRAIN_SEG]
test_probas_labeled_users = test_probas[y_test_labeld_user.index]

In [742]:
y_test_labeld_user

0      0
1      0
2      0
3      0
4      0
      ..
995    9
996    9
997    9
998    9
999    9
Length: 1000, dtype: int64

In [743]:
y_pred_seg = np.array([0 if t in p else 1 for t, p in zip(y_test_labeld_user, get_top_k_classes(test_probas_labeled_users, k=5))])
print(y_pred_seg.sum())
y_pred_seg.shape

917


(1000,)

In [738]:
y_test_labeled = y[:LABELED_USERS*TRAIN_SEG]

(500,)

In [725]:
get_metrics(y_test_labeled, y_pred_seg)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').